### Project Dashboard developed in Bokeh

run in terminal from root directory<p>

```
bokeh serve --show project_dashboard.ipynb
```

In [99]:
import json
from bokeh.io import show, output_file, output_notebook
from bokeh.models import (CDSView, ColorBar, ColumnDataSource,
                          CustomJS, CustomJSFilter, 
                          GeoJSONDataSource, HoverTool,
                          LinearColorMapper, Slider, Select, RadioButtonGroup, Column, Text, LinearAxis, Grid, TapTool)
from bokeh.layouts import column, row, widgetbox
from bokeh.palettes import brewer
from bokeh.plotting import figure

from bokeh.io import curdoc, output_notebook
from bokeh.models import Slider, HoverTool
from bokeh.layouts import widgetbox, row, column

import geopandas as gpd
import pandas as pd
import numpy as np
import os

import logging
  

In [66]:
# loading chemichal data dataset 
chem_data = pd.read_csv(r'assets/ChemDataForJeffOlson.csv', parse_dates=['VisitDate'])
chem_data['year'] = chem_data.VisitDate.dt.year

seasons = {
    '0':'winter',
    '1':'spring',
    '2':'summer',
    '3':'fall'
}

chem_data['season'] =  (chem_data.VisitDate.dt.month - 1) //3
chem_data['season'] = chem_data.season.apply(lambda x: seasons[str(x)])
chem_data['date'] = chem_data['VisitDate'].apply(lambda x:x.strftime("%Y-%m-01"))

In [67]:
"""# set treshold for number of measurments
TRESHOLD = 100

def get_frequent(df, TRESHOLD):
    counts = df.groupby(['CharacteristicID'])['Result'].count()
    mask = counts[counts.values <= TRESHOLD].index
    df = df.where(~df.isin(mask))
    return df
              
chem_data = get_frequent(chem_data, TRESHOLD)"""

"# set treshold for number of measurments\nTRESHOLD = 100\n\ndef get_frequent(df, TRESHOLD):\n    counts = df.groupby(['CharacteristicID'])['Result'].count()\n    mask = counts[counts.values <= TRESHOLD].index\n    df = df.where(~df.isin(mask))\n    return df\n              \nchem_data = get_frequent(chem_data, TRESHOLD)"

In [68]:
df = chem_data.groupby(['LakeID', 'year', 'CharacteristicID'])['Result'].agg(np.mean)
df = df.reset_index()

In [69]:
gdf = gpd.read_file(r'assets/Lake_Inventory_Watersheds.geojson')

In [164]:
def column_data(selectedYear,CharacteristicID):
    yr = selectedYear
    df_yr = df[(df['year'] == yr) & (df['CharacteristicID'] == CharacteristicID)]
    
    result_min = df[df.CharacteristicID == CharacteristicID].Result.min()
    result_max = df[df.CharacteristicID == CharacteristicID].Result.max()
    
    merged = gdf.merge(df_yr, left_on='LAKEID', right_on='LakeID', how = 'left')
    merged.fillna('No data', inplace = True)
    
    return data, result_min, result_max


def json_data(selectedYear,CharacteristicID):
    
    yr = selectedYear
    df_yr = df[(df['year'] == yr) & (df['CharacteristicID'] == CharacteristicID)]
    
    result_min = df[df.CharacteristicID == CharacteristicID].Result.min()
    result_max = df[df.CharacteristicID == CharacteristicID].Result.max()
    
    merged = gdf.merge(df_yr, left_on='LAKEID', right_on='LakeID', how = 'left')
    merged.fillna('No data', inplace = True)


    merged_json = json.loads(merged.to_json())
    json_data = json.dumps(merged_json)


    return json_data, result_min, result_max

def time_series_data(CharacteristicID, index):
 
    LakeID = gdf.iloc[index].LAKEID
    df_Lake = df[(df['CharacteristicID'] == CharacteristicID) & (df['LakeID'] == LakeID)]

    
    return df_Lake

In [165]:
#Input GeoJSON source that contains features for plotting.
geosource=GeoJSONDataSource(geojson=json_data(1980, "TP")[0])

In [166]:
source=ColumnDataSource(time_series_data("TP", 0))

#Define a sequential multi-hue color palette.
palette = brewer["YlGnBu"][8]

#Reverse color order so that dark blue is highest obesity.
palette = palette[::-1]

#Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors. Input nan_color.
color_mapper = LinearColorMapper(palette = palette, low = 0, high = 40, nan_color = "#d9d9d9")

#Create color bar.
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8,width = 20, height = 550,
                    border_line_color=None,location = (0,0), orientation = "vertical")

#Add hover tool
hover = HoverTool(tooltips = [ ("Lake","@LakeID"),("Result", "@Result")])

tap = TapTool(behavior='select', gesture='tap')

# Create a figure object for line chart

time_series = figure(title = 'Chemichal data time-series', 
           plot_height = 130,
           plot_width = 500, 
           toolbar_location = None)
time_series.xgrid.visible = False
time_series.xaxis.major_tick_line_color = None  
time_series.xaxis.minor_tick_line_color = None
time_series.yaxis.minor_tick_line_color = None  
time_series.xaxis.major_label_text_font_size = '0pt'  


time_series.scatter(source = source, x='year', y='Result')


#Create figure object for geomap.
geo = figure(title = "Vermont Lay monitoring programm",
           plot_height = 600,
           plot_width = 500,
           toolbar_location = None,
           tools = [hover, tap, 'pan', 'wheel_zoom'],
           x_axis_location=None,
           y_axis_location=None,
          )


geo.xgrid.grid_line_color = None
geo.ygrid.grid_line_color = None

# Add patch renderer to figure.
geo.patches("xs","ys",
          source = geosource,
          fill_color = {"field" :"Result",
                        "transform" : color_mapper},
          line_color = "black",
          line_width = 0.25,
          fill_alpha = 1)



# Define the callback function: update_plot
def update_plot(attr, old, new):
    
    yr = slider.value
    CharacteristicID = parameter_select.value
    
    new_data, color_mapper.low, color_mapper.high = json_data(yr, CharacteristicID)
    
    geosource.geojson = new_data
    geo.title.text = "Vermont Lay monitoring programm, {}\n Parammeter: {}".format(yr,CharacteristicID)

    

# Define a callback function: update_time_series
def update_time_series(attr, old, new):
    
    selections = new
    CharacteristicID = parameter_select.value
    log = logging.getLogger('bokeh')
    log.info(type(new), type(new[0]))
    #log.info(selections_2)
    new_data = time_series_data(CharacteristicID, selections[0])
    source.data = new_data
    
    
    
# Make a slider object: slider 
slider = Slider(title = "Year",start = 1980, end = 2021, step = 1, value = 1980, width=500)
slider.on_change("value",update_plot)

# Make a drop down object: selector 
LABELS = list(df.CharacteristicID.unique())
parameter_select = Select(value="TP", options=LABELS)
parameter_select.on_change("value", update_plot)
    
    
geosource.selected.on_change('indices', update_time_series)




#Specify layout
geo.add_layout(color_bar, "right")

first_row = row(geo, time_series)
second_row = row(widgetbox(slider, parameter_select))


# Make a column layout of widgetbox(slider) and plot, and add it to the current document
layout = column(first_row, second_row)
curdoc().add_root(layout)